In [1]:
from instituto_langchain.automatize_me.graphs.revisor_codigo_graph import RevisorCodigoGraph

In [2]:
revisor_graph = RevisorCodigoGraph()

2025-04-15 15:17:15,665 - INFO - Inicializando GerenciadorModelo
2025-04-15 15:17:16,797 - INFO - LLM DEEPSEEK inicializado (modelo: deepseek-r1-distill-llama-70b, temperatura: 0.5)
2025-04-15 15:17:17,299 - INFO - LLM GEMINI inicializado (modelo: gemini-2.0-flash, temperatura: 0.5)


In [3]:
CONTROLLER = """
package br.com.brconsorcios.workflow.domain.reativacaocotas.application;

import static br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotasConverter.toAbrirAtividadeCommand;
import static br.com.brconsorcios.workflow.util.ValidacoesUtil.requireCollectionNonNullOrEmpty;
import static br.com.brconsorcios.workflow.util.ValidacoesUtil.requireClassNonNull;
import static br.com.brconsorcios.workflow.util.ValidacoesUtil.isValidClass;

import java.util.List;
import java.util.Map;

import br.com.brconsorcios.servicos.domain.reserva.core.model.response.ResponseReservarCota;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotas;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotasConsorciadoCotas;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.EmailAtividadeConferirBoletoCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.EmailAtividadeReativarCotaCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.EmailAtividadeVerificarTermosCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.ReativacaoCotasConsorciadoCotasCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.ReativacaoCotasReservarCotaCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.VerificarPagamentoCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.dados.ReativacaoCotasEmailDados;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotaVerificarPagamento;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotasAbrirAtividade;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotasConsultaAtividades;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotasConsultarConsorciadoCotas;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotasReservarCota;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.SendEmailAtividadeConferirBoleto;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.SendEmailAtividadeReativarCota;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.SendEmailAtividadeVerificarTermos;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.outgoing.ReativacaoCotasDataBase;
import br.com.brconsorcios.workflow.util.LoggerUtil;

public class ReativacaoCotasController {
    
    private LoggerUtil loggerUtil;
    private ReativacaoCotasDataBase dataBase;

    private ReativacaoCotasAbrirAtividade abrirAtividade;
    private ReativacaoCotasConsultaAtividades consultaAtividades;
    private SendEmailAtividadeConferirBoleto sendEmailAtividadeConferirBoleto;
    private SendEmailAtividadeVerificarTermos sendEmailAtividadeVerificarTermos;
    private SendEmailAtividadeReativarCota sendEmailAtividadeReativarCotaCommand;
    private ReativacaoCotasReservarCota reservarCota;
    private ReativacaoCotasConsultarConsorciadoCotas consultarConsorciadoCotas;
    private ReativacaoCotaVerificarPagamento reativacaoCotaVerificarPagamento;

    public ReativacaoCotasController(
        LoggerUtil loggerUtil,
        ReativacaoCotasAbrirAtividade abrirAtividade,
        ReativacaoCotasConsultaAtividades consultaAtividades,
        SendEmailAtividadeConferirBoleto sendEmailAtividadeConferirBoleto,
        SendEmailAtividadeVerificarTermos sendEmailAtividadeVerificarTermos,
        SendEmailAtividadeReativarCota sendEmailAtividadeReativarCotaCommand,
        ReativacaoCotasReservarCota reservarCota,
        ReativacaoCotasConsultarConsorciadoCotas consultarConsorciadoCotas,
        ReativacaoCotaVerificarPagamento reativacaoCotaVerificarPagamento,
        ReativacaoCotasDataBase dataBase
        ) {

        this.abrirAtividade = abrirAtividade;
        this.consultaAtividades = consultaAtividades;
        this.sendEmailAtividadeConferirBoleto = sendEmailAtividadeConferirBoleto;
        this.sendEmailAtividadeVerificarTermos = sendEmailAtividadeVerificarTermos;
        this.sendEmailAtividadeReativarCotaCommand = sendEmailAtividadeReativarCotaCommand;
        this.reservarCota = reservarCota;
        this.loggerUtil = loggerUtil;
        this.consultarConsorciadoCotas = consultarConsorciadoCotas;
        this.reativacaoCotaVerificarPagamento = reativacaoCotaVerificarPagamento;
        this.dataBase = dataBase;
    }
   
    /*
     * Método responsável por abrir a atividade pública de reativação de cotas
     */
    public ReativacaoCotas abrirAtividadePublica(Map<String, String[]> request){
        requireCollectionNonNullOrEmpty(request, "request");

        loggerUtil.info("Iniciando a criação da instância do processo Reativação de Cotas.");
        loggerUtil.debug("Request recebido: " + request);

        var abrirAtividadeCommand = toAbrirAtividadeCommand(request);
        loggerUtil.debug("Command instânciar processo: " + abrirAtividadeCommand);

        var reativacaoCotas = abrirAtividade.handle(abrirAtividadeCommand);
        loggerUtil.info("Finalizado a criação da instância do processo Reativação de Cotas.");

        return reativacaoCotas;
    }

    /*
     * Método responsável por consultar os processos que estão pendentes
     * de envio de e-mails
     */
    public ReativacaoCotasEmailDados consultarAtividadeParaEnviarEmail() {
        return consultaAtividades.consultarAtividadesEmail();
    }

    /*
     * Método responsável por consultar os processos que estão parados
     * na atividae Reservar Cotas
     */
    public List<ReativacaoCotas> consultarAtividadesReservarCotas() {
        return dataBase.obterAtividadesReservarCotas();
    }

    /*
     * Método responsável por consultar os processos que estão parados
     * na Verificar Pagamento
     */
    public List<ReativacaoCotas> consultarInstanciasVerificarPagamento() {
        return dataBase.obterAtividadeVerificarPagamento();
    }

    /*
     * Verifica se o consórciado pagou a parcela no prazo correto
     */
    public void verificarPagamentoParcelaReativacao(VerificarPagamentoCommand command) {
        requireClassNonNull(command, "VerificarPagamentoCommand"); 
        isValidClass(command);

        loggerUtil.info("Workflow Domain Reativação de Cotas");
        loggerUtil.info("   Application - Iniciando a verificação do pagamento da parcela");
        loggerUtil.info("   Application - Command: " + command.toString());
        reativacaoCotaVerificarPagamento.verificarPagamento(command);
    }

    /*
     * Envia e-mail, e atualiza a atividade para que não envio novamente. 
     */
    public void enviarEmailAtividadeConferirBoleto(EmailAtividadeConferirBoletoCommand emailAtividadeConferirBoletoCommand) {
        requireClassNonNull(emailAtividadeConferirBoletoCommand, "EmailAtividadeConferirBoletoCommand");
        isValidClass(emailAtividadeConferirBoletoCommand);        
     
        sendEmailAtividadeConferirBoleto.handle(emailAtividadeConferirBoletoCommand);
    }

    /*
     * Envia e-mail, e atualiza a atividade para que não envio novamente. 
     */
    public void enviarEmailAtividadeVerificarTermo(EmailAtividadeVerificarTermosCommand emailAtividadeVerificarTermosCommand) {
        requireClassNonNull(emailAtividadeVerificarTermosCommand, "EmailAtividadeVerificarTermosCommand");
        isValidClass(emailAtividadeVerificarTermosCommand);        
     
        sendEmailAtividadeVerificarTermos.handle(emailAtividadeVerificarTermosCommand);
    }

    /*
     * Envia e-mail para o Consorciado informando sobre a finalização do processo
     */
    public void enviarEmailAtividadeReativarCota(EmailAtividadeReativarCotaCommand emailAtividadeReativarCotaCommand){
        requireClassNonNull(emailAtividadeReativarCotaCommand, "EmailAtividadeReativarCotaCommand"); 
        isValidClass(emailAtividadeReativarCotaCommand);

        sendEmailAtividadeReativarCotaCommand.handle(emailAtividadeReativarCotaCommand);
    }

    /*
     * Método responsável por reservar a cota
     */    
    public ResponseReservarCota reservarCota(ReativacaoCotasReservarCotaCommand reservarCotaCommand){
        requireClassNonNull(reservarCotaCommand, "ReativacaoCotasReservarCotaCommand"); 
        isValidClass(reservarCotaCommand);

        loggerUtil.info("Workflow Domain Reativação de Cotas");
        loggerUtil.info("   Application - Iniciando a reserva da cota");
        loggerUtil.info("   Application - Command: " + reservarCotaCommand.toString());
        return reservarCota.handle(reservarCotaCommand);
    }

    public ReativacaoCotasConsorciadoCotas obterConsorciadoCotas(ReativacaoCotasConsorciadoCotasCommand consorciadoCotasCommand){ 
        requireClassNonNull(consorciadoCotasCommand, "ReativacaoCotasConsorciadoCotasCommand");
        isValidClass(consorciadoCotasCommand);
        
        System.out.println(consorciadoCotasCommand);
        loggerUtil.info("Iniciando a consulta do consorciado e suas cotas.");
        loggerUtil.debug("Command Consorciado Cotas " + consorciadoCotasCommand);
       
        var consoricadoCotas = consultarConsorciadoCotas.handle(consorciadoCotasCommand);
        loggerUtil.info("Finalizado a consulta do consorciado e suas cotas.");

        return consoricadoCotas;
    }
}
"""

In [4]:
FACADE = """
package br.com.brconsorcios.workflow.domain.reativacaocotas.core;

import static br.com.brconsorcios.workflow.domain.reativacaocotas.core.ReativacaoCotasFacadeSupport.*;
import static br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotasConverter.toReativacaoCotasAtividadeEmailDto;
import static br.com.brconsorcios.workflow.util.data.DataUtil.getStringDataAtual;

import java.util.ArrayList;
import java.util.Comparator;
import java.util.Optional;
import java.util.stream.Collectors;

import br.com.brconsorcios.servicos.domain.cotaservice.core.model.CotasClienteEntidadeServico;
import br.com.brconsorcios.servicos.domain.pessoa.application.PessoaController;
import br.com.brconsorcios.servicos.domain.pessoa.core.model.command.PessoaConsultarEmailCommand;
import br.com.brconsorcios.servicos.domain.pessoa.core.model.response.EmailEntidadeServico;
import br.com.brconsorcios.servicos.domain.reserva.core.model.response.ResponseReservarCota;
import br.com.brconsorcios.workflow.domain.atividade.core.model.CamposAtividade;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotas;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotasConsorciadoCotas;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotasConverter;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.EmailAtividadeConferirBoletoCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.EmailAtividadeReativarCotaCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.EmailAtividadeVerificarTermosCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.ReativacaoCotasConsorciadoCotasCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.ReativacaoCotasInstanciarProcessoCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.ReativacaoCotasReservarCotaCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.command.VerificarPagamentoCommand;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.dados.ReativacaoCotasEmailDados;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.exception.ReativacaoCotasClienteIrregularException;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.exception.ReativacaoCotasDataMaximaReservaException;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.exception.ReativacaoCotasGrupoSemVagasException;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.exception.ReativacaoCotasNaoPagouParcelaException;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotaVerificarPagamento;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotasAbrirAtividade;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotasConsultaAtividades;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotasConsultarConsorciadoCotas;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.ReativacaoCotasReservarCota;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.SendEmailAtividadeConferirBoleto;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.SendEmailAtividadeReativarCota;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.incoming.SendEmailAtividadeVerificarTermos;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.outgoing.ReativacaoCotasDataBase;
import br.com.brconsorcios.workflow.domain.reativacaocotas.infrastructure.ReativacaoCotasInstanciaProcessoService;
import br.com.brconsorcios.workflow.domain.usuario.core.model.TipoUsuario;
import br.com.brconsorcios.workflow.util.LoggerUtil;

public class ReativacaoCotasFacade implements 
    ReativacaoCotasAbrirAtividade, 
    ReativacaoCotasConsultaAtividades,
    SendEmailAtividadeConferirBoleto,
    SendEmailAtividadeVerificarTermos,
    SendEmailAtividadeReativarCota,
    ReativacaoCotasReservarCota, 
    ReativacaoCotasConsultarConsorciadoCotas,
    ReativacaoCotaVerificarPagamento
    {

    private LoggerUtil loggerUtil;
    private ReativacaoCotasInstanciaProcessoService instanciaProcessoService;
    private ReativacaoCotasDataBase dataBase;
    private PessoaController pessoaController; //--> Remover, passar pasar Service
    private ReativacaoCotasVerificarPagamentoService verificarPagamentoService;
    private ReativacaoCotasEmailService emailService;
    private ReativacaoCotasCotaService cotaService;
    private ReativacaoCotasReativacaoService reativacaoService;
    private ReativacaoCotasReservaService reservaService;
   
    public ReativacaoCotasFacade(
        LoggerUtil loggerUtil,
        ReativacaoCotasInstanciaProcessoService instanciaProcessoService,
        ReativacaoCotasDataBase dataBase, 
        PessoaController pessoaController,
        ReativacaoCotasVerificarPagamentoService verificarPagamentoService,
        ReativacaoCotasEmailService emailService,
        ReativacaoCotasCotaService cotaService,
        ReativacaoCotasReativacaoService reativacaoService,
        ReativacaoCotasReservaService reservaService
        ) {

        this.instanciaProcessoService = instanciaProcessoService;
        this.dataBase = dataBase;
        this.pessoaController = pessoaController;
        this.loggerUtil = loggerUtil; 
        this.verificarPagamentoService = verificarPagamentoService;
        this.emailService = emailService;
        this.cotaService = cotaService;
        this.reativacaoService = reativacaoService;
        this.reservaService = reservaService;
    }

    @Override
    public ReativacaoCotas handle(ReativacaoCotasInstanciarProcessoCommand command) {   
        loggerUtil.info("Iniciando o processamento do Command Instânciar Processo.");        
        
        var instanciarProcessoCampos = instanciaProcessoService.criarCamposAberturaInstancia(command);
        var instanciaProcessoUsuario = TipoUsuario.PUBLICO;
        var instanciaReativacaoCotas = instanciaProcessoService.instanciarProcesso(instanciarProcessoCampos, instanciaProcessoUsuario);
        loggerUtil.info("Instância criada.");
       
        instanciaProcessoService.aprovarAtividadeInstancia(instanciaReativacaoCotas, instanciarProcessoCampos, instanciaProcessoUsuario);
        loggerUtil.info("Atividade Instância aprovada.");

        instanciaReativacaoCotas.setCodigoAtividade(ReativacaoCotas.ATIVIDADE_SOLICITAR_REATIVACAO_START);
        var uuid = instanciaProcessoService.obterDadosAtividadeSolicitarStart(instanciaReativacaoCotas);
        instanciaReativacaoCotas.setUuid(uuid);
        loggerUtil.info("UUID da atividade obtido.");
       
        return instanciaReativacaoCotas;
    }

    @Override
    public ReativacaoCotasEmailDados consultarAtividadesEmail() {
        //Consulta todas as atividades para as quais devemos mandar e-mail
        var atividadesEmail = dataBase.getAtividadesEmail();        
        
        /*
         * Atividade: Conferir boleto
         * E-mail: Link de acesso à atividade
         * Regra: Deve ser enviado e-mail para todos
         * processos que estão na atividade Conferir Boleto, 4,
         * cujo campo SN_EMAIL_ENVIADO seja 'Não' 
         */
        var emailAtividadeConferirBoleto = atividadesEmail.stream()
            .filter(isAtividadeConferirBoleto.and(isEmailAtividadeEnviado))
            .collect(Collectors.toList());

        /*
         * Atividade: Verificar Termos
         * E-mail: Link de acesso à atividade
         * Regra: Deve ser enviado e-mail para todos
         * processos que estão na atividade Verificar Termos, 6,
         * cujo campo SN_EMAIL_ENVIADO seja 'Não' 
         */
        var emailAtividadeVerificarTermos = atividadesEmail.stream()
            .filter(isAtividadeVerificarTermos.and(isEmailAtividadeEnviado))
            .collect(Collectors.toList());

        return new ReativacaoCotasEmailDados(
            toReativacaoCotasAtividadeEmailDto(emailAtividadeConferirBoleto),
            toReativacaoCotasAtividadeEmailDto(emailAtividadeVerificarTermos)
        );
    }

    @Override
    public void handle(EmailAtividadeVerificarTermosCommand command) {
        loggerUtil.info("Iniciando o processamento do Command Email Atividade Verificar Termos.");
        
        emailService.enviarEmailVerificarTermo(command);
        loggerUtil.info("E-Mail da atividade enviado.");
       
        var dadosAtividade = instanciaProcessoService.criarDadoAtividade(command);
        var camposAtualizacaoEmail = instanciaProcessoService.criarCamposAtualizacaoEmail();
        var tipoUsuario = TipoUsuario.PUBLICO;
        instanciaProcessoService.atualizarInstancia(dadosAtividade, camposAtualizacaoEmail, tipoUsuario);
        loggerUtil.info("Atividade atualizada.");
        
    }

    @Override
    public void handle(EmailAtividadeConferirBoletoCommand command) {
        loggerUtil.info("Iniciando o processamento do Command Email Atividade Conferir Boleto.");
        
        emailService.enviarEmailConferirBoleto(command); 
        loggerUtil.info("E-Mail da atividade enviado.");

        var camposAtualizacaoEmail = instanciaProcessoService.criarCamposAtualizacaoEmail();
        var dadosAtividade = instanciaProcessoService.criarDadoAtividade(command);
        var tipoUsuario = TipoUsuario.PUBLICO;
        instanciaProcessoService.atualizarInstancia(dadosAtividade, camposAtualizacaoEmail, tipoUsuario);
        loggerUtil.info("Atividade atualizada.");
    }

    @Override
    /*
     * Envia e-mail de finalização do processo
     */
    public void handle(EmailAtividadeReativarCotaCommand command) {
        loggerUtil.info("Iniciando o processamento do Command Email Atividade Reativar Cota.");

        emailService.enviarEmailReativarCota(command); 
        loggerUtil.info("E-Mail da atividade enviado.");
    }

    @Override
    /*
     * Verifica se a cota que está na instância do processo pagou a parcela dentro do prazo esperado
     * Se não pagou, e não tem prazo -> Atualiza a instância, falando que não pagou, cancela o processo e envia um e-mail informando o não pagamento.
     * Se não pagou, e ainda há prazo para pagar -> Atualiza a instância falando que não pagou.
     * Se pagou -> Atualiza a instância falando que pagou, e aprova.
     */
    public void verificarPagamento(VerificarPagamentoCommand command) {
        loggerUtil.info("Iniciando o processamento do Command Verificar Pagamento.");
        
        var camposAtividade = new ArrayList<CamposAtividade>();
        var tipoUsuario = TipoUsuario.ROBO;
        var atividade = instanciaProcessoService.criarDadoAtividade(command);
        
        loggerUtil.info("Verificando se ainda existe prazo válido para analisar o pagamento.");
        if (!verificarPagamentoService.dentroPrazoReativacao(command)) {
            camposAtividade.add(new CamposAtividade("PC_PAGA", "NÃO"));
            camposAtividade.add(new CamposAtividade("DS_ROBO_PC_PAGA", "Consorciado não pagou a parcela dentro do prazo esperado."));

            loggerUtil.info("Tentando atualizar a instância da atividade.");
            instanciaProcessoService.atualizarInstancia(atividade, camposAtividade, tipoUsuario);

            loggerUtil.info("Tentando aprovar a instância do processo.");
            instanciaProcessoService.aprovarAtividadeInstancia(atividade, camposAtividade, tipoUsuario);
            
            loggerUtil.info("Tentando enviar e-mail informando sobre o cancelamento da instância.");
            emailService.enviarEmailFinalizacaoFaltaPagamento(command);

            throw new ReativacaoCotasDataMaximaReservaException();
        } 
        
        if (!verificarPagamentoService.pagouParcelaReativacao(command)) {
            camposAtividade.add(new CamposAtividade("PC_PAGA", "NÃO"));
            camposAtividade.add(new CamposAtividade("DS_ROBO_PC_PAGA", "Consorciado não pagou a parcela."));

            loggerUtil.info("Tentando atualizar a atividade.");
            instanciaProcessoService.atualizarInstancia(atividade, camposAtividade, tipoUsuario);           

            throw new ReativacaoCotasNaoPagouParcelaException();
        }
        
        var dataPagamento = getStringDataAtual();
        camposAtividade.add(new CamposAtividade("PC_PAGA", "SIM"));
        camposAtividade.add(new CamposAtividade("DS_ROBO_PC_PAGA", "Consorciado pagou a parcela."));
        camposAtividade.add(new CamposAtividade("DATA_PAGAMENTO", dataPagamento));

        loggerUtil.info("Tentando atualizar a atividade.");
        instanciaProcessoService.atualizarInstancia(atividade, camposAtividade, tipoUsuario);           

        loggerUtil.info("Tentando atualizar e aprovar a instância do processo a atividade.");
        instanciaProcessoService.aprovarAtividadeInstancia(atividade, camposAtividade, tipoUsuario);
    }
    
    @Override    
    public ResponseReservarCota handle(ReativacaoCotasReservarCotaCommand reservarCotaCommand) {
        loggerUtil.info("Iniciando o processamento do Command Reservar Cota.");
       
        loggerUtil.info("Obtendo as cotas do consorciado");
        var cotasConsorciado = cotaService.consultarCotasCliente(reservarCotaCommand.getInscricaoNacional(), reservarCotaCommand.getIdUsuario());

        loggerUtil.info("Obtendo as informações detalhadas de cada cota");
        var informacoesCota = cotaService.consultarInformacoesCota(cotasConsorciado);

        var atividade = instanciaProcessoService.criarDadoAtividade(reservarCotaCommand);
        var tipoUsuario = TipoUsuario.ROBO;

        loggerUtil.info("Buscando Cotas IRREGURALARES");
         var cotasIrregulares = informacoesCota.stream()
            .filter(isVersaoZero.and(isCotaDiferenteQuitada).and(isCotaDiferenteNormal))
            .collect(Collectors.toList());
        
        if (!cotasIrregulares.isEmpty()) {
            loggerUtil.info("Consorciado IRREGURALAR");
           
            var campoAtividadeConsorciadoIrregular = instanciaProcessoService.criarCamposConsorciadoIrregular(cotasIrregulares);
        
            loggerUtil.info("Atualizando a instância");
            instanciaProcessoService.atualizarInstancia(atividade, campoAtividadeConsorciadoIrregular, tipoUsuario);

            loggerUtil.info("Encaminhando a instância ao departamento de sustentação");
            instanciaProcessoService.aprovarAtividadeInstancia(atividade, campoAtividadeConsorciadoIrregular, tipoUsuario);            
            
            throw new ReativacaoCotasClienteIrregularException();
        }

        loggerUtil.info("Procurando por vagas no grupo");
        var vagasGrupo = reativacaoService.consultarVagasGrupo(reservarCotaCommand.getIdGrupo());
        
        if (vagasGrupo.isEmpty()) {
            loggerUtil.info("Grupo SEM VAGA");
            var campoAtividadeGrupoSemVaga = instanciaProcessoService.criarCamposGrupoSemVaga();
            
            loggerUtil.info("Atualizando a instância");
            instanciaProcessoService.atualizarInstancia(atividade, campoAtividadeGrupoSemVaga, tipoUsuario);
            
            throw new ReativacaoCotasGrupoSemVagasException();
        } 

        loggerUtil.info("Consorciado REGULAR e grupo COM VAGA");
        loggerUtil.info("Reservando a cota");
        var cotaDisponivel = vagasGrupo.get(0).getCodigoCota();
        var cotaReservada = reservaService.realizarReserva(reservarCotaCommand.getIdGrupo(), cotaDisponivel);

        var campoAtividadeCotaReservada = instanciaProcessoService.criarCamposCotaReservada(cotaReservada, reservarCotaCommand.getCodigoGrupo(), String.valueOf(cotaDisponivel));

        loggerUtil.info("Atualizando a instância");
        instanciaProcessoService.atualizarInstancia(atividade, campoAtividadeCotaReservada, tipoUsuario);

        loggerUtil.info("Encaminhando a instância para conferência da reserva");
        instanciaProcessoService.aprovarAtividadeInstancia(atividade, campoAtividadeCotaReservada, tipoUsuario); 
       
        return cotaReservada;
    }

    @Override
    public ReativacaoCotasConsorciadoCotas handle(ReativacaoCotasConsorciadoCotasCommand consorciadoCotasCommand) {
        var builderRetorno = ReativacaoCotasConsorciadoCotas.builder();

        loggerUtil.info("Tentando consultar cotas do consorciado");
        var cotasConsorciado = cotaService.consultarCotasCliente(consorciadoCotasCommand.getInscricaoNacional(), consorciadoCotasCommand.getIdUsuario());
       
        loggerUtil.info("Tentando consultar informações das cotas");
        var informacoesCota = cotaService.consultarInformacoesCota(cotasConsorciado);

        // Só deve retornar cotas com versão entre 1 e 39
        var cotasReativacao = informacoesCota.stream()
            .filter(isCotaReativacao)
            .map(ReativacaoCotasConverter::toReativacaoCotasCotas)
            .collect(Collectors.toList());

        builderRetorno.cotasConsorciado(cotasReativacao);
        
        loggerUtil.info("Tentando consultar e-mail do consorciado");

        // Pega o ID_Pessoa do maior ID_Cota, esse é o ID_Pessoa da última cota do consorciado
        Optional<Integer> idPessoa = cotasConsorciado.stream()
            .max(Comparator.comparing(CotasClienteEntidadeServico::getIdCota))
            .map(CotasClienteEntidadeServico::getIdPessoa);

        var pessoaCommand = PessoaConsultarEmailCommand.builder()
            .idPessoa(idPessoa.get())
            .build();
        var emailsConsorciado = pessoaController.consultarEmail(pessoaCommand);

        // Pega o E-Mail do maior ID_Email, esse é o último e-mail adicionado no cadastro de pessoas
        Optional<String> emailConsorciado = emailsConsorciado.stream()
            .max(Comparator.comparing(EmailEntidadeServico::getIdEmail))
            .map(EmailEntidadeServico::getEmail);
        builderRetorno.emailConsorciado(emailConsorciado.get());

        // Pega o nome do consorciado da última cota que ele comprou
        Optional<String> nomeConsorciado = cotasConsorciado.stream()
            .max(Comparator.comparing(CotasClienteEntidadeServico::getIdCota))
            .map(CotasClienteEntidadeServico::getNomePessoa);        
        builderRetorno.nomeConsorciado(nomeConsorciado.get());

        // Retorna
        return builderRetorno.build();
    }
}
"""

In [5]:
INFRASTRUCTURE = """
package br.com.brconsorcios.workflow.domain.reativacaocotas.infrastructure;

import static br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotas.ATIVIDADE_CONFERIR_BOLETO;
import static br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotas.ATIVIDADE_RESERVAR_COTA;
import static br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotas.ATIVIDADE_VERIFICAR_TERMOS;
import static br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotas.ATIVIDADE_AGUARDAR_PAGAMENTO;
import static br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotas.NOME_TABELA;

import java.sql.SQLException;
import java.util.ArrayList;
import java.util.List;
import java.util.Map;
import java.util.stream.Collectors;

import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotas;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.ReativacaoCotasConverter;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.model.exception.ReativacaoCotasConsultaException;
import br.com.brconsorcios.workflow.domain.reativacaocotas.core.ports.outgoing.ReativacaoCotasDataBase;
import br.com.brconsorcios.workflow.exception.InfrastrutureException;
import br.com.brconsorcios.workflow.infrastructure.db.workflow.WorkflowRepository;
import br.com.brconsorcios.workflow.util.SelectAtividadeUtil;

public class ReativacaoCotasDataBaseAdapter implements ReativacaoCotasDataBase{

    private WorkflowRepository workflowRepository;
    private final SelectAtividadeUtil BUILDER_SELECT;

    public ReativacaoCotasDataBaseAdapter(WorkflowRepository workflowRepository) {
        this.workflowRepository = workflowRepository;
       
        //Constrói o builde do select que será usado como base para as consultas
        this.BUILDER_SELECT = new SelectAtividadeUtil(NOME_TABELA);
    }
    

    @Override
    public List<ReativacaoCotas> getAtividadesEmail() {
        //Constrói a consulta SQL para buscar atividades   
        var select = queryAtividadeEmail();   
       
        //Executa a consulta e obtém o resultado
        List<Map<String, Object>> atividade = new ArrayList<>();
        try {
            atividade.addAll(workflowRepository.selectMany(select, null));

            //Converte o resultado em uma lista de objetos do domínio            
            return atividade.stream()
                .map(ReativacaoCotasConverter::toReativacaoCotas)
                .collect(Collectors.toList());
                
        } catch (SQLException e) {            
            throw new ReativacaoCotasConsultaException("Erro ao consultar atividades para enviar e-mail - " + e.getMessage()  + "\n" + select, e);
        }
    }
    
    /*
     * Consulta processos que estão paradas nas atividade publicas, são elas:
     * Conferir Boleto (Start)
     * Verificar Termos (Start)
     */
    private String queryAtividadeEmail(){
        var query = 
            BUILDER_SELECT
                .select("TABELA_PROCESSO.SN_EMAIL_ENVIADO ")
                .select("TABELA_PROCESSO.EMAIL_SOLICITANTE ")
                .select("TABELA_PROCESSO.SCP_SOLICITANTE ")                
                .where("PROCESSO.IDE_FINALIZADO = 'A'")//Consulta somente atividades ativas
                .addWhere("PROCESSO.COD_ETAPA_ATUAL IN ("
                    + ATIVIDADE_CONFERIR_BOLETO + ", "                  
                    + ATIVIDADE_VERIFICAR_TERMOS + ")")                
                .criarQuery();
            
        return query;
    }

    @Override
    public List<ReativacaoCotas> obterAtividadesReservarCotas() {
        //Constrói a consulta SQL para buscar atividades   
        var select = queryAtividadesReservarCotas();   
       
        //Executa a consulta e obtém o resultado
        List<Map<String, Object>> atividade = new ArrayList<>();
        try {
            atividade.addAll(workflowRepository.selectMany(select, null));

            //Converte o resultado em uma lista de objetos do domínio            
            return atividade.stream()
                .map(ReativacaoCotasConverter::toReativacaoCotasReservarCotas)
                .collect(Collectors.toList());
                
        } catch (InfrastrutureException e) {            
            return new ArrayList<ReativacaoCotas>();
        }  catch (SQLException e) {            
            throw new ReativacaoCotasConsultaException("Erro ao consultar atividades para reservar cota - " + e.getMessage()  + "\n" + select, e);
        }
    }
    
    private String queryAtividadesReservarCotas() {
        var query =
            BUILDER_SELECT
                .select("TABELA_PROCESSO.GRUPO")
                .select("TABELA_PROCESSO.COTA")
                .select("TABELA_PROCESSO.ID_GRUPO")
                .select("TABELA_PROCESSO.VERSAO_CONSORCIADO")
                .select("TABELA_PROCESSO.CPF_CONSORCIADO")
                .where("PROCESSO.IDE_FINALIZADO = 'A'")
                .addWhere("PROCESSO.COD_ETAPA_ATUAL IN ("
                    + ATIVIDADE_RESERVAR_COTA + ")")                
                .criarQuery();

        return query;
    }

    @Override
    public List<ReativacaoCotas> obterAtividadeVerificarPagamento() {
        //Constrói a consulta SQL para buscar atividades   
        var select = queryInstanciasVerificarPagamento();   
       
        //Executa a consulta e obtém o resultado
        List<Map<String, Object>> atividade = new ArrayList<>();
        try {
            atividade.addAll(workflowRepository.selectMany(select, null));

            //Converte o resultado em uma lista de objetos do domínio            
            return atividade.stream()
                .map(ReativacaoCotasConverter::toReativacaoCotasVerificarPagamento)
                .collect(Collectors.toList());
                
        } catch (InfrastrutureException e) {            
            return new ArrayList<ReativacaoCotas>();
        } catch (SQLException e) {            
            throw new ReativacaoCotasConsultaException("Erro ao consultar instâncias da atividade Aguardar Pagamento - " + e.getMessage()  + "\n" + select, e);
        }
    }

    private String queryInstanciasVerificarPagamento() {
        var builderSelect = new SelectAtividadeUtil(NOME_TABELA);
        var query =
            builderSelect
                .select("TABELA_PROCESSO.ID_COTA ")
                .select("TABELA_PROCESSO.GRUPO ")
                .select("TABELA_PROCESSO.COTA ")
                .select("TABELA_PROCESSO.VERSAO_CONSORCIADO ")
                .select("TABELA_PROCESSO.EMAIL_SOLICITANTE ")
                .select("TABELA_PROCESSO.SCP_SOLICITANTE ")
                .select("TABELA_PROCESSO.PRORROGAR AS DT_MAXIMA_RESERVA ")
                .select("TABELA_PROCESSO.CODIGO_RESERVA ")
                .select("TABELA_PROCESSO.DT_RESERVA ")
                .where("PROCESSO.IDE_FINALIZADO = 'A' ")//Consulta somente atividades ativas
                .addWhere("PROCESSO.COD_ETAPA_ATUAL IN ("
                    + ATIVIDADE_AGUARDAR_PAGAMENTO + ") ")
                .orderBy("PROCESSO.COD_PROCESSO ")
                .orderDescendente()
                .criarQuery();

        return query;
    }
}

"""

In [6]:
report = await revisor_graph.executar_review({
    "Controller": CONTROLLER,
    "Facade": FACADE,
    "Infrastructure": INFRASTRUCTURE,
})

In [ ]:
print(report['final_report'])

In [8]:
# Save the value of report['final_report'] to a file
with open("../.files/final_report.md", "w", encoding="utf-8") as file:
    file.write(report['final_report'])

# Save the value of report['final_report'] to a file
with open("../.files/quality_review.md", "w", encoding="utf-8") as file:
    file.write(report['quality_review'])

with open("../.files/design_review.md", "w", encoding="utf-8") as file:
    file.write(report['design_review'])